<a href="https://colab.research.google.com/github/MehrdadDastouri/hotel_management_system/blob/main/hotel_management_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile

# مسیر فایل فشرده Test.zip
zip_path = "/content/Test.zip"

# پوشه‌ای برای استخراج فایل‌ها
extract_path = "/content/Test"

# استخراج فایل‌ها
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print("فایل‌ها با موفقیت استخراج شدند.")

# نمایش فایل‌های استخراج‌شده
print("\nلیست فایل‌های موجود در پوشه تست:")
for root, dirs, files in os.walk(extract_path):
    for file in files:
        file_path = os.path.join(root, file)
        print(f"- {file}")

# خواندن و نمایش 5 خط اول از هر فایل متنی
print("\nمحتوای فایل‌های تست (فقط 5 خط اول):")
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".txt"):  # فقط فایل‌های متنی
            file_path = os.path.join(root, file)
            print(f"\n--- محتوای فایل: {file} (5 خط اول) ---")
            with open(file_path, 'r', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    print(line.strip())  # حذف فاصله‌های اضافی
                    if i == 4:  # فقط 5 خط اول را نمایش دهد
                        break

فایل‌ها با موفقیت استخراج شدند.

لیست فایل‌های موجود در پوشه تست:
- test_cancelled.txt
- test_blockuser.txt
- test_room_stats.txt
- test_reservation.txt
- test_user.txt
- test_roomclean.txt
- test_room.txt

محتوای فایل‌های تست (فقط 5 خط اول):

--- محتوای فایل: test_cancelled.txt (5 خط اول) ---
regestercode,reason
1020,انصراف مشتری
1021,مشکل فنی

--- محتوای فایل: test_blockuser.txt (5 خط اول) ---
userid,blockdate
1111111111,2025-01-15
2222222222,2025-01-16

--- محتوای فایل: test_room_stats.txt (5 خط اول) ---
room_id,reservations_count,service_counter,total_reserved_days
111,2,6,10
112,1,3,5
113,3,5,15
221,4,7,20

--- محتوای فایل: test_reservation.txt (5 خط اول) ---
userid,startdate,roomid,enddate,regestercode
1234567890,2025-01-20,122,2025-01-25,1001
2345678901,2025-01-22,128,2025-01-27,1002
3456789012,2025-01-23,136,2025-01-28,1003
4567890123,2025-01-24,138,2025-01-29,1004

--- محتوای فایل: test_user.txt (5 خط اول) ---
userid,password
5497814515,NoULj8Ny8
4055534741,puNvmIhUeV
94211743

In [ ]:
# ساختمان داده‌های اصلی

# کلاس Node: نود عمومی برای لیست پیوندی، درخت AVL و غیره
class Node:
    def __init__(self, key=None, value=None):
        self.key = key
        self.value = value
        self.height = 1  # برای درخت AVL
        self.left = None
        self.right = None
        self.next = None  # برای لیست پیوندی

# کلاس Array: آرایه پیوندی
class Array:
    def __init__(self, size):
        self.size = size
        self.head = None
        current = None
        for i in range(size):
            new_node = Node(i)
            if not self.head:
                self.head = new_node
            else:
                current.next = new_node
            current = new_node

    def __getitem__(self, index):
        if 0 <= index < self.size:
            current = self.head
            for _ in range(index):
                current = current.next
            return current.value
        raise IndexError("Index out of range")

    def __setitem__(self, index, value):
        if 0 <= index < self.size:
            current = self.head
            for _ in range(index):
                current = current.next
            current.value = value
        else:
            raise IndexError("Index out of range")

    def __len__(self):
        return self.size

# کلاس Stack: پشته (لیست پیوندی)
class Stack:
    def __init__(self):
        self.top = None
        self.size = 0

    def is_empty(self):
        return self.top is None

    def push(self, key, value=None):
        new_node = Node(key, value)
        new_node.next = self.top
        self.top = new_node
        self.size += 1

    def pop(self):
        if self.is_empty():
            return None, None
        temp = self.top
        self.top = self.top.next
        self.size -= 1
        return temp.key, temp.value

    def peek(self):
        if self.is_empty():
            return None, None
        return self.top.key, self.top.value

In [ ]:
class Queue:
    def __init__(self, mode="auto"):
        self.front = None
        self.rear = None
        self.size = 0
        self.mode = mode

    def dequeue(self):
        if self.is_empty():
            return None

        temp = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        self.size -= 1

        if isinstance(temp.value, Reservation):
            return temp.key, temp.value
        elif isinstance(temp.value, tuple):
            return temp.value
        else:
            return temp.key, temp.value

    def is_empty(self):
        return self.front is None

    def enqueue(self, key, value=None):
        if value is None:
            value = key
            if hasattr(value, 'national_id'):
                key = value.national_id
            elif hasattr(value, 'room_id'):
                key = value.room_id
            elif hasattr(value, 'reservation_id'):
                key = value.reservation_id
            else:
                key = id(value)

        new_node = Node(key, value)
        if self.is_empty():
            self.front = self.rear = new_node
        else:
            self.rear.next = new_node
            self.rear = new_node
        self.size += 1

    def peek(self):
        if self.is_empty():
            return (None, None) if self.mode == "tuple" else None

        if self.mode == "tuple":
            if isinstance(self.front.value, tuple):
                return self.front.value
            return self.front.key, self.front.value
        elif self.mode == "object":
            return self.front.value
        else:  # mode == "auto"
            if isinstance(self.front.value, (Room, User, Reservation)):
                return self.front.value
            if isinstance(self.front.value, tuple):
                return self.front.value
            return self.front.key, self.front.value

    def __iter__(self):
        current = self.front
        while current:
            if self.mode == "tuple":
                if isinstance(current.value, tuple):
                    yield current.value
                else:
                    yield current.key, current.value
            elif self.mode == "object":
                yield current.value
            else:  # mode == "auto"
                if isinstance(current.value, (Room, User, Reservation)):
                    yield current.value
                elif isinstance(current.value, tuple):
                    yield current.value
                else:
                    yield current.key, current.value
            current = current.next

    def __len__(self):
        return self.size

    def clear(self):
        self.front = None
        self.rear = None
        self.size = 0

    def to_list(self):
        result = []
        for item in self:
            result.append(item)
        return result

    def copy(self):
        new_queue = Queue(mode=self.mode)
        current = self.front
        while current:
            new_queue.enqueue(current.key, current.value)
            current = current.next
        return new_queue

    def change_mode(self, new_mode):
        if new_mode not in ["tuple", "object", "auto"]:
            raise ValueError("Invalid mode")
        self.mode = new_mode

In [ ]:
class AVLTree:
    def __init__(self):
        self.root = None
        self.traversal_queue = None

    def height(self, node):
        if not node:
            return 0
        return node.height

    def update_height(self, node):
        if node:
            node.height = max(self.height(node.left), self.height(node.right)) + 1

    def balance_factor(self, node):
        if not node:
            return 0
        return self.height(node.left) - self.height(node.right)

    def right_rotate(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        self.update_height(y)
        self.update_height(x)
        return x

    def left_rotate(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        self.update_height(x)
        self.update_height(y)
        return y

    def get_min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current

    def insert(self, key, value=None):
        def _insert(root, key, value):
            if not root:
                return Node(key, value)
            if key < root.key:
                root.left = _insert(root.left, key, value)
            elif key > root.key:
                root.right = _insert(root.right, key, value)
            else:
                root.value = value
                return root

            self.update_height(root)
            balance = self.balance_factor(root)

            if balance > 1:
                if key < root.left.key:
                    return self.right_rotate(root)
                if key > root.left.key:
                    root.left = self.left_rotate(root.left)
                    return self.right_rotate(root)

            if balance < -1:
                if key > root.right.key:
                    return self.left_rotate(root)
                if key < root.right.key:
                    root.right = self.right_rotate(root.right)
                    return self.left_rotate(root)

            return root

        self.root = _insert(self.root, key, value)

    def search(self, key):
        def _search(root, key):
            if not root or root.key == key:
                return root
            if key < root.key:
                return _search(root.left, key)
            return _search(root.right, key)

        result = _search(self.root, key)
        return result.value if result else None

    def inorder_traversal(self, mode="auto"):
        self.traversal_queue = Queue(mode=mode)

        def _inorder(node):
            if node:
                _inorder(node.left)
                if isinstance(node.value, Reservation):
                    self.traversal_queue.enqueue(node.key, node.value)
                else:
                    self.traversal_queue.enqueue(node.key, node.value)
                _inorder(node.right)

        _inorder(self.root)
        return self.traversal_queue

    def get_all_values(self):
        queue = Queue(mode="auto")

        def _traverse(node):
            if node:
                _traverse(node.left)
                queue.enqueue(node.key, node.value)
                _traverse(node.right)

        _traverse(self.root)
        return queue

    def delete(self, key):
        def _delete(root, key):
            if not root:
                return root

            if key < root.key:
                root.left = _delete(root.left, key)
            elif key > root.key:
                root.right = _delete(root.right, key)
            else:
                if not root.left:
                    return root.right
                elif not root.right:
                    return root.left

                temp = self.get_min_value_node(root.right)
                root.key = temp.key
                root.value = temp.value
                root.right = _delete(root.right, temp.key)

            self.update_height(root)
            balance = self.balance_factor(root)

            if balance > 1:
                if self.balance_factor(root.left) >= 0:
                    return self.right_rotate(root)
                root.left = self.left_rotate(root.left)
                return self.right_rotate(root)

            if balance < -1:
                if self.balance_factor(root.right) <= 0:
                    return self.left_rotate(root)
                root.right = self.right_rotate(root.right)
                return self.left_rotate(root)

            return root

        self.root = _delete(self.root, key)

    def get_range(self, start_key, end_key):
        result_queue = Queue(mode="tuple")

        def _range_traversal(node):
            if not node:
                return
            if node.key > start_key:
                _range_traversal(node.left)
            if start_key <= node.key <= end_key:
                if isinstance(node.value, Reservation):
                    reservation = node.value
                    result_queue.enqueue(
                        node.key,
                        (reservation.user_id,
                         reservation.checkin,
                         reservation.room_id,
                         reservation.checkout,
                         reservation.reservation_id)
                    )
                else:
                    result_queue.enqueue(node.key, node.value)
            if node.key < end_key:
                _range_traversal(node.right)

        _range_traversal(self.root)
        return result_queue

In [ ]:
class HashTable:
    def __init__(self, size=1009):
        self.size = size
        self.table = Array(size)
        self.count = 0

    def hash_function(self, key):
        if isinstance(key, str):
            hash_value = 0
            for char in key:
                hash_value = (hash_value * 31 + ord(char)) % self.size
            return hash_value
        elif isinstance(key, (int, float)):
            return int(key) % self.size
        elif isinstance(key, tuple):
            hash_value = 0
            for item in key:
                if isinstance(item, (int, float)):
                    hash_value = (hash_value * 31 + int(item)) % self.size
                elif isinstance(item, str):
                    for char in item:
                        hash_value = (hash_value * 31 + ord(char)) % self.size
            return hash_value
        return 0

    def insert(self, key, value):
        index = self.hash_function(key)
        new_node = Node(key, value)

        if not self.table[index]:
            self.table[index] = new_node
        else:
            current = self.table[index]
            while current:
                if current.key == key:
                    current.value = value
                    return
                if not current.next:
                    break
                current = current.next
            current.next = new_node
        self.count += 1

    def search(self, key):
        index = self.hash_function(key)
        current = self.table[index]

        while current:
            if current.key == key:
                return current.value
            current = current.next
        return None

    def get_all_values(self, mode="tuple"):
        result_queue = Queue(mode=mode)
        for i in range(self.size):
            current = self.table[i]
            while current:
                result_queue.enqueue(current.key, current.value)
                current = current.next
        return result_queue

    def delete(self, key):
        index = self.hash_function(key)
        current = self.table[index]
        prev = None

        while current:
            if current.key == key:
                if prev:
                    prev.next = current.next
                else:
                    self.table[index] = current.next
                self.count -= 1
                return True
            prev = current
            current = current.next
        return False

    def __len__(self):
        return self.count

    def update(self, key, value):
        current = self.table[self.hash_function(key)]
        while current:
            if current.key == key:
                current.value = value
                return True
            current = current.next
        return False

In [ ]:
class User:
    def __init__(self, national_id, password):
        self.national_id = national_id
        self.password = password
        self.blocked_until = None
        self.reservations_count = 0
        self.cancelled_count = 0

class Room:
    def __init__(self, room_id, floor, beds, room_num):
        self.room_id = room_id
        self.floor = floor
        self.beds = beds
        self.room_num = room_num
        self.active = True
        self.reservations_count = 0        # تعداد کل رزروها
        self.total_reserved_days = 0       # مجموع کل روزهای رزرو شده
        self.service_counter = 0           # شمارنده روزها برای سرویس
        self.cleaning_status = True        # وضعیت نظافت
        self.last_cleaned = None           # آخرین تاریخ نظافت
        self.next_active_date = None       # تاریخ فعال شدن بعد از سرویس

class Reservation:
    def __init__(self, reservation_id, user_id, room_id, checkin, checkout):
        self.reservation_id = reservation_id
        self.user_id = user_id
        self.room_id = room_id
        self.checkin = checkin
        self.checkout = checkout
        self.is_cancelled = False
        self.cancel_reason = None
        self.user_info = None

In [ ]:
from datetime import datetime, timedelta

class DataStructure:
    def __init__(self, policy_manager=None):
        self.users = HashTable()
        self.rooms = HashTable()
        self.reservations = AVLTree()
        self.cleaning_queue = Queue()
        self.blocked_users = HashTable()
        self.cancelled_reservations = HashTable()
        self.reservation_history = AVLTree()
        self.policy_manager = policy_manager

###سلول 2 - متدهای مدیریتی:


In [ ]:
def get_all_rooms(self):
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        result.enqueue(room.room_id, room)
    return result

def get_inactive_rooms(self):
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if not room.active:
            result.enqueue(room.room_id, room)
    return result

def get_all_users(self):
    result = Queue()
    users_queue = self.users.get_all_values()
    while not users_queue.is_empty():
        _, user = users_queue.dequeue()
        result.enqueue(user.national_id, user)
    return result

def add_room(self, floor, beds, room_num):
    if not (1 <= beds <= 5):
        raise ValueError("تعداد تخت باید بین 1 تا 5 باشد")

    room_id = int(f"{floor}{beds}{room_num}")
    if self.rooms.search(room_id):
        return room_id

    room = Room(room_id, floor, beds, room_num)
    self.rooms.insert(room_id, room)
    return room_id

def deactivate_room(self, room_id):
    room = self.get_room(room_id)
    if not room:
        raise ValueError("اتاق مورد نظر یافت نشد")
    room.active = False
    self.rooms.update(room_id, room)
    return True

def activate_room(self, room_id):
    room = self.get_room(room_id)
    if not room:
        raise ValueError("اتاق مورد نظر یافت نشد")
    room.active = True
    self.rooms.update(room_id, room)
    return True

def get_blocked_users(self):
    blocked_queue = Queue()
    blocked_values = self.blocked_users.get_all_values()

    while not blocked_values.is_empty():
        user_id, block_date = blocked_values.dequeue()
        blocked_queue.enqueue(user_id, block_date)

    return blocked_queue

def get_cleaning_rooms(self):
    result = Queue()
    rooms_queue = self.rooms.get_all_values()

    print("\nوضعیت اتاق‌ها:")
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        print(f"اتاق {room.room_id}: service_counter={room.service_counter}")  # حذف active از خروجی

        if room.service_counter >= 10 or (not room.active and room.service_counter > 0):
            result.enqueue(room.room_id, room)

    return result

def service_room(self, room_id):
    room = self.get_room(room_id)
    if not room:
        return False

    room.next_active_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
    room.service_counter = 0
    self.rooms.update(room_id, room)
    return True

def process_cleaning_queue(self):
    if not self.cleaning_queue.is_empty():
        room_id = self.cleaning_queue.dequeue()[0]
        room = self.get_room(room_id)
        if room:
            room.cleaning_status = True
            room.last_cleaned = datetime.now().strftime("%Y-%m-%d")
            self.rooms.update(room_id, room)
            return room_id
    return None

def process_date_change(self, current_date):
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        room_id, room = rooms_queue.dequeue()

        if room.next_active_date and room.next_active_date <= current_date:
            room.active = True
            room.next_active_date = None
            self.rooms.update(room_id, room)

        if room.last_cleaned:
            days_since_cleaning = (datetime.strptime(current_date, "%Y-%m-%d") -
                                 datetime.strptime(room.last_cleaned, "%Y-%m-%d")).days
            if days_since_cleaning >= 3:
                self.add_to_cleaning_queue(room_id)

def get_cancelled_reservations(self):
    result = Queue()
    cancelled_queue = self.cancelled_reservations.get_all_values()
    while not cancelled_queue.is_empty():
        reservation_id, reservation = cancelled_queue.dequeue()
        result.enqueue(reservation_id, reservation)
    return result

def unblock_user(self, user_id):
    self.blocked_users.delete(user_id)
    return True

####سلول 3 - متدهای پنل مشتری:



In [ ]:
def find_rooms_in_floor_range(self, start_floor, end_floor):
    """جستجوی اتاق در بازه طبقات"""
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if start_floor <= room.floor <= end_floor and room.active:
            result.enqueue(room.room_id, room)
    return result

def get_sorted_rooms(self, sort_type, floor=None, beds=None):
    """مرتب‌سازی اتاق‌ها با فیلترهای مختلف"""
    result = Queue()
    temp_queue = Queue()
    rooms_queue = self.rooms.get_all_values()

    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if not room.active:
            continue

        if floor is not None and beds is not None:
            if room.floor == floor and room.beds == beds:
                temp_queue.enqueue(room.room_id, room)
        elif floor is not None:
            if room.floor == floor:
                temp_queue.enqueue(room.room_id, room)
        elif beds is not None:
            if room.beds == beds:
                temp_queue.enqueue(room.room_id, room)
        else:
            temp_queue.enqueue(room.room_id, room)

    while not temp_queue.is_empty():
        current_id, current_room = temp_queue.dequeue()
        temp_sort = Queue()
        inserted = False

        while not result.is_empty():
            existing_id, existing_room = result.dequeue()

            if not inserted:
                if sort_type == "floor" and current_room.floor < existing_room.floor:
                    temp_sort.enqueue(current_id, current_room)
                    inserted = True
                elif sort_type == "beds" and current_room.beds < existing_room.beds:
                    temp_sort.enqueue(current_id, current_room)
                    inserted = True

            temp_sort.enqueue(existing_id, existing_room)

        if not inserted:
            temp_sort.enqueue(current_id, current_room)

        result = temp_sort

    return result

def check_room_availability(self, room_id, checkin, checkout):
    """بررسی در دسترس بودن اتاق برای رزرو"""
    room = self.get_room(room_id)
    if not room or not room.active:
        return False

    checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
    checkout_date = datetime.strptime(checkout, "%Y-%m-%d")

    current_reservations = Queue()
    reservations_queue = self.reservations.inorder_traversal()

    while not reservations_queue.is_empty():
        _, reservation = reservations_queue.dequeue()
        if reservation.room_id == room_id and not reservation.is_cancelled:
            current_reservations.enqueue(reservation.reservation_id, reservation)

    while not current_reservations.is_empty():
        _, reservation = current_reservations.dequeue()
        existing_checkin = datetime.strptime(reservation.checkin, "%Y-%m-%d")
        existing_checkout = datetime.strptime(reservation.checkout, "%Y-%m-%d")
        if not (checkout_date <= existing_checkin or checkin_date >= existing_checkout):
            return False

    return True

def add_reservation(self, reservation_id, user_id, room_id, checkin, checkout, ignore_room_status=False):
    """افزودن رزرو جدید"""
    # بررسی مسدود نبودن کاربر
    if self.is_user_blocked(user_id):
        raise ValueError("حساب کاربری شما مسدود است")

    # بررسی وجود و وضعیت اتاق
    room = self.get_room(room_id)
    if not room:
        raise ValueError("اتاق مورد نظر یافت نشد")

    if not ignore_room_status and not room.active:
        raise ValueError("اتاق مورد نظر غیرفعال است")

    # بررسی تکراری نبودن کد رزرو
    if self.reservations.search(reservation_id):
        raise ValueError(f"رزرو با کد {reservation_id} قبلاً ثبت شده است")

    # بررسی معتبر بودن تاریخ‌ها
    try:
        checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
        checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
        if checkin_date >= checkout_date:
            raise ValueError("تاریخ ورود باید قبل از تاریخ خروج باشد")
    except ValueError as e:
        if "time data" in str(e):
            raise ValueError("فرمت تاریخ نامعتبر است")
        raise e

    # بررسی تداخل با رزروهای موجود
    current_reservations = self.get_room_reservations(room_id)
    while not current_reservations.is_empty():
        _, existing_reservation = current_reservations.dequeue()
        if not existing_reservation.is_cancelled:
            existing_checkin = datetime.strptime(existing_reservation.checkin, "%Y-%m-%d")
            existing_checkout = datetime.strptime(existing_reservation.checkout, "%Y-%m-%d")
            if not (checkout_date <= existing_checkin or checkin_date >= existing_checkout):
                raise ValueError("تداخل در تاریخ رزرو")

    # ایجاد و ذخیره رزرو جدید
    reservation = Reservation(reservation_id, user_id, room_id, checkin, checkout)
    self.reservations.insert(reservation_id, reservation)
    self.reservation_history.insert(reservation_id, reservation)

    # به‌روزرسانی آمار اتاق
    room.reservations_count += 1
    room.total_reserved_days += (checkout_date - checkin_date).days
    self.rooms.update(room_id, room)

    # به‌روزرسانی آمار کاربر
    user = self.get_user(user_id)
    if user:
        user.reservations_count += 1
        self.users.update(user_id, user)

    # بررسی نیاز به نظافت
    if room.total_reserved_days >= 3:
        self.add_to_cleaning_queue(room_id)

    return True

def get_most_reserved_rooms(self, beds):
    """دریافت اتاق‌های با بیشترین رزرو"""
    rooms_queue = self.rooms.get_all_values()
    most_reserved = Queue()
    max_reservations = 0

    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if room.beds == beds and room.active:
            if room.reservations_count > max_reservations:
                max_reservations = room.reservations_count
                most_reserved = Queue()
                most_reserved.enqueue(room.room_id, room)
            elif room.reservations_count == max_reservations:
                most_reserved.enqueue(room.room_id, room)

    return most_reserved

def add_user(self, national_id, password):
    if not isinstance(national_id, str) or len(national_id) != 10:
        raise ValueError("شماره ملی باید 10 رقم باشد")
    if self.users.search(national_id):
        raise ValueError(f"کاربر با کد ملی {national_id} قبلاً ثبت شده است")

    user = User(national_id, password)
    self.users.insert(national_id, user)
    return True

def get_user_active_reservations(self, user_id):
    result = Queue()
    reservations_queue = self.reservations.inorder_traversal()
    while not reservations_queue.is_empty():
        _, reservation = reservations_queue.dequeue()
        if reservation.user_id == user_id and not reservation.is_cancelled:
            result.enqueue(reservation.reservation_id, reservation)
    return result


def cancel_reservation(self, reservation_id, reason):
    """لغو رزرو با محدودیت یک روز قبل"""
    reservation = self.reservations.search(reservation_id)
    if not reservation:
        raise ValueError("رزرو مورد نظر یافت نشد")

    if reservation.is_cancelled:
        raise ValueError("این رزرو قبلاً لغو شده است")

    checkin_date = datetime.strptime(reservation.checkin, "%Y-%m-%d")
    if (checkin_date - datetime.now()).days < 1:
        raise ValueError("لغو رزرو باید حداقل یک روز قبل از تاریخ ورود انجام شود")

    room = self.get_room(reservation.room_id)
    if room:
        checkout_date = datetime.strptime(reservation.checkout, "%Y-%m-%d")
        days = (checkout_date - checkin_date).days
        room.total_reserved_days -= days
        room.reservations_count -= 1
        if not room.active and room.total_reserved_days <= 10:
            room.active = True
        self.rooms.update(reservation.room_id, room)

    user = self.get_user(reservation.user_id)
    if user:
        user.cancelled_count += 1
        user.reservations_count -= 1
        self.users.update(reservation.user_id, user)

    reservation.is_cancelled = True
    reservation.cancel_reason = reason
    self.cancelled_reservations.insert(reservation_id, reservation)
    self.reservations.delete(reservation_id)
    return True

def get_user_reservation_history(self, user_id):
    result = Queue()
    # استفاده از AVLTree برای تاریخچه
    history_queue = self.reservation_history.inorder_traversal()
    cancelled_queue = self.cancelled_reservations.get_all_values()

    # افزودن رزروهای فعال
    while not history_queue.is_empty():
        _, reservation = history_queue.dequeue()
        if reservation.user_id == user_id:
            result.enqueue(reservation.reservation_id, reservation)

    # افزودن رزروهای لغو شده
    while not cancelled_queue.is_empty():
        reservation_id, reservation = cancelled_queue.dequeue()
        if reservation.user_id == user_id:
            result.enqueue(reservation_id, reservation)

    return result

def auto_reserve_room(self, user_id, beds, checkin, checkout):
    """رزرو خودکار اتاق"""
    available_rooms = Queue()
    rooms_queue = self.rooms.get_all_values()

    # بررسی محدودیت زمانی
    checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
    checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
    days = (checkout_date - checkin_date).days

    if days < 1 or days > 7:
        raise ValueError("مدت اقامت باید بین 1 تا 7 روز باشد")

    # جستجو در AVLTree اتاق‌ها
    while not rooms_queue.is_empty():
        room_id, room = rooms_queue.dequeue()
        if room.beds == beds and room.active:
            if self.check_room_availability(room_id, checkin, checkout):
                available_rooms.enqueue(room_id, room)

    if not available_rooms.is_empty():
        room_id, room = available_rooms.dequeue()
        reservation_id = int(f"{user_id}{room_id}{datetime.now().strftime('%H%M%S')}")

        if self.add_reservation(reservation_id, user_id, room_id, checkin, checkout):
            # تغییر وضعیت اتاق به غیرفعال در AVLTree
            room.active = False
            self.rooms.update(room_id, room)
            return room_id, reservation_id

    return None, None

def get_best_rooms(self, beds):
    """دریافت سه اتاق با بیشترین رزرو"""
    result = Queue()
    temp_queue = Queue()
    rooms_queue = self.rooms.get_all_values()

    # جمع‌آوری اتاق‌های مناسب در یک صف موقت
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if room.beds == beds and room.active:
            temp_queue.enqueue(room.reservations_count, room)

    # انتخاب سه اتاق با بیشترین رزرو
    for _ in range(3):
        if temp_queue.is_empty():
            break

        max_count = -1
        max_room = None
        temp_store = Queue()

        while not temp_queue.is_empty():
            count, room = temp_queue.dequeue()
            if count > max_count:
                if max_room:
                    temp_store.enqueue(max_count, max_room)
                max_count = count
                max_room = room
            else:
                temp_store.enqueue(count, room)

        if max_room:
            result.enqueue(max_room.room_id, max_room)
        temp_queue = temp_store

    return result

###سلول 4 - متدهای کمکی و مشترک:



In [ ]:
def get_user(self, national_id):
    return self.users.search(national_id)

def get_room(self, room_id):
    return self.rooms.search(room_id)

def is_user_blocked(self, national_id):
    return self.blocked_users.search(national_id) is not None

def get_room_reservations(self, room_id):
    result = Queue()
    reservations_queue = self.reservations.inorder_traversal()
    while not reservations_queue.is_empty():
        _, reservation = reservations_queue.dequeue()
        if reservation.room_id == room_id:
            result.enqueue(reservation.reservation_id, reservation)
    return result

def get_active_rooms(self):
    """دریافت اتاق‌های فعال"""
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if room.active:
            result.enqueue(room.room_id, room)
    return result

def get_sorted_rooms_by_beds(self):
    rooms_queue = self.get_active_rooms()
    result = Queue()

    while not rooms_queue.is_empty():
        current_id, current_room = rooms_queue.dequeue()
        temp_sort = Queue()
        inserted = False

        while not result.is_empty():
            existing_id, existing_room = result.dequeue()
            if not inserted and current_room.beds < existing_room.beds:
                temp_sort.enqueue(current_id, current_room)
                inserted = True
            temp_sort.enqueue(existing_id, existing_room)

        if not inserted:
            temp_sort.enqueue(current_id, current_room)

        result = temp_sort

    return result

def get_sorted_rooms_by_floor(self):
    rooms_queue = self.get_active_rooms()
    result = Queue()

    while not rooms_queue.is_empty():
        current_id, current_room = rooms_queue.dequeue()
        temp_sort = Queue()
        inserted = False

        while not result.is_empty():
            existing_id, existing_room = result.dequeue()
            if not inserted and current_room.floor < existing_room.floor:
                temp_sort.enqueue(current_id, current_room)
                inserted = True
            temp_sort.enqueue(existing_id, existing_room)

        if not inserted:
            temp_sort.enqueue(current_id, current_room)

        result = temp_sort

    return result

def check_reservation_date_range(self, checkin, checkout):
    """بررسی محدودیت‌های تاریخ رزرو"""
    checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
    checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
    days = (checkout_date - checkin_date).days

    if days < 1 or days > 7:
        raise ValueError("مدت رزرو باید بین 1 تا 7 روز باشد")
    return True

def check_room_total_reserved_days(self, room_id, days):
    """بررسی محدودیت 10 روز رزرو"""
    room = self.get_room(room_id)
    if not room:
        raise ValueError("اتاق مورد نظر یافت نشد")

    total_days = room.total_reserved_days + days
    return total_days <= 10

def add_to_cleaning_queue(self, room_id):
    room = self.get_room(room_id)
    if not room:
        raise ValueError("اتاق مورد نظر یافت نشد")

    temp_queue = Queue()
    is_duplicate = False

    while not self.cleaning_queue.is_empty():
        current_room_id = self.cleaning_queue.dequeue()[0]
        if current_room_id == room_id:
            is_duplicate = True
        temp_queue.enqueue(current_room_id)

    while not temp_queue.is_empty():
        self.cleaning_queue.enqueue(temp_queue.dequeue()[0])

    if not is_duplicate:
        self.cleaning_queue.enqueue(room_id)
        room.cleaning_status = False
        self.rooms.update(room_id, room)
        return True
    return False

def get_reservations_in_range(self, start_date, end_date):
    """دریافت رزروها در یک بازه زمانی مشخص"""
    result = Queue()
    try:
        target_start = datetime.strptime(start_date, "%Y-%m-%d")
        target_end = datetime.strptime(end_date, "%Y-%m-%d")

        reservations_queue = self.reservations.inorder_traversal()

        while not reservations_queue.is_empty():
            _, reservation = reservations_queue.dequeue()
            reservation_start = datetime.strptime(reservation.checkin, "%Y-%m-%d")
            reservation_end = datetime.strptime(reservation.checkout, "%Y-%m-%d")

            if (reservation_start >= target_start and
                reservation_end <= target_end):
                result.enqueue(reservation.reservation_id, reservation)

    except Exception as e:
        print(f"خطا در بررسی بازه زمانی: {e}")

    return result


def find_rooms_by_beds(self, beds):
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if room.beds == beds and room.active:
            result.enqueue(room.room_id, room)
    return result

def find_rooms_by_floor(self, floor):
    result = Queue()
    rooms_queue = self.rooms.get_all_values()
    while not rooms_queue.is_empty():
        _, room = rooms_queue.dequeue()
        if room.floor == floor and room.active:
            result.enqueue(room.room_id, room)
    return result

In [ ]:
# تخصیص تمام توابع به کلاس DataStructure
DataStructure.get_all_rooms = get_all_rooms
DataStructure.get_inactive_rooms = get_inactive_rooms
DataStructure.get_all_users = get_all_users
DataStructure.add_room = add_room
DataStructure.deactivate_room = deactivate_room
DataStructure.activate_room = activate_room
DataStructure.get_blocked_users = get_blocked_users
DataStructure.get_cleaning_rooms = get_cleaning_rooms
DataStructure.service_room = service_room
DataStructure.process_cleaning_queue = process_cleaning_queue
DataStructure.process_date_change = process_date_change
DataStructure.get_cancelled_reservations = get_cancelled_reservations
DataStructure.unblock_user = unblock_user
DataStructure.find_rooms_in_floor_range = find_rooms_in_floor_range
DataStructure.get_sorted_rooms = get_sorted_rooms
DataStructure.check_room_availability = check_room_availability
DataStructure.add_reservation = add_reservation
DataStructure.get_most_reserved_rooms = get_most_reserved_rooms
DataStructure.add_user = add_user
DataStructure.get_user_active_reservations = get_user_active_reservations
DataStructure.cancel_reservation = cancel_reservation
DataStructure.get_user_reservation_history = get_user_reservation_history
DataStructure.auto_reserve_room = auto_reserve_room
DataStructure.get_best_rooms = get_best_rooms
DataStructure.get_room = get_room
DataStructure.get_user = get_user
DataStructure.is_user_blocked = is_user_blocked
DataStructure.get_room_reservations = get_room_reservations
DataStructure.get_active_rooms = get_active_rooms
DataStructure.get_sorted_rooms_by_beds = get_sorted_rooms_by_beds
DataStructure.get_sorted_rooms_by_floor = get_sorted_rooms_by_floor
DataStructure.check_reservation_date_range = check_reservation_date_range
DataStructure.check_room_total_reserved_days = check_room_total_reserved_days
DataStructure.add_to_cleaning_queue = add_to_cleaning_queue
DataStructure.get_reservations_in_range = get_reservations_in_range
DataStructure.find_rooms_by_beds = find_rooms_by_beds
DataStructure.find_rooms_by_floor = find_rooms_by_floor

In [ ]:
import os
from datetime import datetime

class FileManager:
    def __init__(self, directory="Test"):
        self.directory = directory
        self.ROOM_STATS_FILE = "test_room_stats.txt"
        self.data_loaded = False  # متغیر کنترلی برای بررسی یکبار لود شدن
        self.ensure_directory_exists()

def load_all_data(self, data_structure):
    if self.data_loaded:  # اگر قبلاً لود شده باشد، دیگر لود نمی‌کنیم
        return

    print("--- بارگذاری داده‌ها از فایل‌ها ---")

    users_path = os.path.join(self.directory, "test_user.txt")
    print(f"Loading users from: {os.path.abspath(users_path)}")
    self.load_users(users_path, data_structure)

    rooms_path = os.path.join(self.directory, "test_room.txt")
    print(f"Loading rooms from: {os.path.abspath(rooms_path)}")
    self.load_rooms(rooms_path, data_structure)

    room_stats_path = os.path.join(self.directory, self.ROOM_STATS_FILE)
    print(f"Loading room statistics from: {os.path.abspath(room_stats_path)}")
    self.load_room_stats(room_stats_path, data_structure)

    reservations_path = os.path.join(self.directory, "test_reservation.txt")
    print(f"Loading reservations from: {os.path.abspath(reservations_path)}")
    self.load_reservations(reservations_path, data_structure)

    blocked_users_path = os.path.join(self.directory, "test_blockuser.txt")
    print(f"Loading blocked users from: {os.path.abspath(blocked_users_path)}")
    self.load_blocked_users(blocked_users_path, data_structure)

    cancelled_path = os.path.join(self.directory, "test_cancelled.txt")
    print(f"Loading cancelled reservations from: {os.path.abspath(cancelled_path)}")
    self.load_cancelled_reservations(cancelled_path, data_structure)

    print("همه اطلاعات با موفقیت بارگذاری شدند.")
    self.data_loaded = True  # پس از بارگذاری موفق، این متغیر را True می‌کنیممی‌کنیم

def ensure_directory_exists(self):
    if not os.path.exists(self.directory):
        os.makedirs(self.directory)
    print(f"Checking directory: {os.path.abspath(self.directory)}\n")


def load_users(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن header
            for line in file:
                if line.strip():
                    try:
                        national_id, password = line.strip().split(',')
                        data_structure.add_user(national_id, password)
                    except Exception as e:
                        print(f"خطا در بارگذاری کاربر: {e}")

def load_rooms(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن header
            for line in file:
                if line.strip():
                    try:
                        room_id, status = line.strip().split(',')
                        room_id = int(room_id)
                        floor = room_id // 100
                        beds = (room_id // 10) % 10
                        room_num = room_id % 10

                        data_structure.add_room(floor, beds, room_num)

                        if status.strip().lower() != "true":
                            data_structure.deactivate_room(room_id)

                    except Exception as e:
                        print(f"خطا در بارگذاری اتاق: {e}")

def load_room_stats(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن سطر هدر
            for line in file:
                if line.strip():
                    try:
                        room_id, reservations_count, service_counter, total_reserved_days = line.strip().split(',')
                        room_id = int(room_id)
                        room = data_structure.rooms.search(room_id)
                        if room:
                            room.reservations_count = int(reservations_count)
                            room.service_counter = int(service_counter)
                            room.total_reserved_days = int(total_reserved_days)
                    except Exception as e:
                        print(f"خطا در بارگذاری آمار اتاق: {e}")

def load_reservations(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن header
            for line in file:
                if line.strip():
                    try:
                        user_id, checkin, room_id, checkout, reservation_id = line.strip().split(',')
                        room_id = int(room_id)
                        reservation_id = int(reservation_id)

                        # ایجاد شیء Reservation
                        reservation = Reservation(
                            reservation_id=reservation_id,
                            user_id=user_id,
                            room_id=room_id,
                            checkin=checkin,
                            checkout=checkout
                        )

                        # افزودن به درخت AVL
                        data_structure.reservations.insert(reservation_id, reservation)

                        # به‌روزرسانی آمار اتاق
                        room = data_structure.rooms.search(room_id)
                        if room:
                            room.reservations_count += 1

                    except Exception as e:
                        print(f"خطا در بارگذاری رزرو: {e}")

def load_blocked_users(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن header
            for line in file:
                if line.strip():
                    try:
                        user_id, block_date = line.strip().split(',')
                        user_id = user_id.strip()
                        block_date = block_date.strip()
                        # حذف این خط print
                        # print(f"Loading blocked user: {user_id}, {block_date}")

                        data_structure.blocked_users.insert(user_id, block_date)

                    except Exception as e:
                        print(f"خطا در بارگذاری کاربر مسدود: {e}")

def load_cancelled_reservations(self, filename, data_structure):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            next(file)  # رد کردن header
            for line in file:
                if line.strip():
                    try:
                        regcode, reason = line.strip().split(',')
                        data_structure.cancelled_reservations.insert(regcode, reason)
                    except Exception as e:
                        print(f"خطا در بارگذاری رزرو لغو شده: {e}")

def save_all_data(self, data_structure):
    print("\n--- ذخیره داده‌ها در فایل‌ها ---")

    try:
        # ذخیره کاربران
        with open(os.path.join(self.directory, "test_user.txt"), 'w', encoding='utf-8') as file:
            file.write("userid,password\n")
            users_queue = data_structure.users.get_all_values()
            while not users_queue.is_empty():
                user_id, user = users_queue.dequeue()
                file.write(f"{user.national_id},{user.password}\n")

        # ذخیره اتاق‌ها
        with open(os.path.join(self.directory, "test_room.txt"), 'w', encoding='utf-8') as file:
            file.write("roomid,status\n")
            rooms_queue = data_structure.rooms.get_all_values()
            while not rooms_queue.is_empty():
                room_id, room = rooms_queue.dequeue()
                status = "True" if room.active else "False"
                file.write(f"{room.room_id},{status}\n")

        # ذخیره آمار اتاق‌ها
        with open(os.path.join(self.directory, self.ROOM_STATS_FILE), 'w', encoding='utf-8') as file:
            file.write("room_id,reservations_count,service_counter,total_reserved_days\n")
            rooms_queue = data_structure.rooms.get_all_values()
            while not rooms_queue.is_empty():
                room_id, room = rooms_queue.dequeue()
                file.write(f"{room_id},{room.reservations_count},{room.service_counter},{room.total_reserved_days}\n")

        # ذخیره رزروها
        with open(os.path.join(self.directory, "test_reservation.txt"), 'w', encoding='utf-8') as file:
            file.write("userid,startdate,roomid,enddate,regestercode\n")
            reservations_queue = data_structure.reservations.inorder_traversal()
            while not reservations_queue.is_empty():
                _, reservation = reservations_queue.dequeue()
                file.write(f"{reservation.user_id},{reservation.checkin},"
                      f"{reservation.room_id},{reservation.checkout},"
                      f"{reservation.reservation_id}\n")

        # ذخیره کاربران مسدود شده
        with open(os.path.join(self.directory, "test_blockuser.txt"), 'w', encoding='utf-8') as file:
            file.write("userid,blockdate\n")
            blocked_queue = data_structure.blocked_users.get_all_values()
            while not blocked_queue.is_empty():
                user_id, block_date = blocked_queue.dequeue()
                file.write(f"{user_id},{block_date}\n")

        # ذخیره رزروهای لغو شده
        with open(os.path.join(self.directory, "test_cancelled.txt"), 'w', encoding='utf-8') as file:
            file.write("regestercode,reason\n")
            cancelled_queue = data_structure.cancelled_reservations.get_all_values()
            while not cancelled_queue.is_empty():
                regcode, reason = cancelled_queue.dequeue()
                file.write(f"{regcode},{reason}\n")

        print("همه اطلاعات با موفقیت ذخیره شدند.")
        return True

    except Exception as e:
        print(f"خطا در ذخیره اطلاعات: {e}")
        return False

# Adding methods to the class
FileManager.ensure_directory_exists = ensure_directory_exists
FileManager.load_all_data = load_all_data
FileManager.load_users = load_users
FileManager.load_rooms = load_rooms
FileManager.load_room_stats = load_room_stats
FileManager.load_reservations = load_reservations
FileManager.load_blocked_users = load_blocked_users
FileManager.load_cancelled_reservations = load_cancelled_reservations
FileManager.save_all_data = save_all_data

In [ ]:
class AdminMenu:
    def __init__(self, data_structure, file_manager, policy_manager):
        self.data_structure = data_structure
        self.file_manager = file_manager
        self.policy_manager = policy_manager

def show_menu(self):
    while True:
        print("\n--- پنل مدیر ---")
        print("1. ایجاد اتاق")
        print("2. نمایش تمام اتاق‌ها")
        print("3. نمایش مشتریان")
        print("4. جستجوی اتاق")
        print("5. مشاهده اتاق‌ها با تعداد تخت مشخص")
        print("6. مشاهده اتاق در یک طبقه مشخص")
        print("7. نمایش اتاق‌های غیر فعال")
        print("8. تغییر وضعیت فعالیت اتاق")
        print("9. مشاهده تاریخچه رزروهای اتاق")
        print("10. مشاهده رزروها به صورت بازه‌ای")
        print("11. مشاهده تمامی رزروهای لغو شده")
        print("12. مشاهده کاربران منع شده")
        print("13. سرویس اتاق")
        print("14. رفتن به روز بعد")
        print("15. خروج")

        choice = input("انتخاب کنید: ")

        try:
            if choice == "1":
                self.create_room()
            elif choice == "2":
                self.display_all_rooms()
            elif choice == "3":
                self.display_all_customers()
            elif choice == "4":
                self.search_room()
            elif choice == "5":
                self.view_rooms_by_beds()
            elif choice == "6":
                self.view_rooms_by_floor()
            elif choice == "7":
                self.display_inactive_rooms()
            elif choice == "8":
                self.change_room_status()
            elif choice == "9":
                self.view_room_history()
            elif choice == "10":
                self.view_reservations_by_date_range()
            elif choice == "11":
                self.view_cancelled_reservations()
            elif choice == "12":
                self.view_blocked_users()
            elif choice == "13":
                self.room_service()
            elif choice == "14":
                self.go_to_next_day()
            elif choice == "15":
                break
            else:
                print("انتخاب نامعتبر. لطفاً دوباره امتحان کنید.")
        except ValueError as e:
            print(f"خطا: {e}")
        except Exception as e:
            print(f"خطای غیرمنتظره: {e}")

def create_room(self):
    try:
        floor = int(input("طبقه: "))
        beds = int(input("تعداد تخت: "))
        room_num = int(input("شماره اتاق: "))

        room_id = self.data_structure.add_room(floor, beds, room_num)
        print(f"اتاق {room_id} با موفقیت ایجاد شد.")
    except ValueError as e:
        print(f"خطا در ایجاد اتاق: {e}")

def display_all_rooms(self):
    rooms = self.data_structure.get_all_rooms()
    if not rooms:
        print("هیچ اتاقی موجود نیست.")
        return

    print("\n--- لیست تمام اتاق‌ها ---")
    for room in rooms:
        status = "فعال" if room.active else "غیرفعال"
        print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت, وضعیت: {status}")

def display_all_customers(self):
    users = self.data_structure.get_all_users()
    if not users:
        print("هیچ مشتری‌ای ثبت نشده است.")
        return

    print("\n--- لیست تمام مشتریان ---")
    for user in users:
        blocked_status = " (مسدود)" if self.data_structure.is_user_blocked(user.national_id) else ""
        print(f"کاربر: {user.national_id}{blocked_status}")

def search_room(self):
    try:
        room_id = int(input("شماره اتاق: "))
        room = self.data_structure.get_room(room_id)

        if room:
            status = "فعال" if room.active else "غیرفعال"
            print(f"\nاتاق {room_id}:")
            print(f"طبقه: {room.floor}")
            print(f"تعداد تخت: {room.beds}")
            print(f"وضعیت: {status}")
            print(f"تعداد رزروها: {room.reservations_count}")
            print(f"مجموع روزهای رزرو شده: {room.total_reserved_days}")
        else:
            print("اتاق مورد نظر یافت نشد.")
    except ValueError:
        print("شماره اتاق باید عدد باشد.")

def view_rooms_by_beds(self):
    try:
        beds = int(input("تعداد تخت مورد نظر: "))
        if not 1 <= beds <= 5:
            print("تعداد تخت باید بین 1 تا 5 باشد")
            return

        print(f"\n--- اتاق‌های با {beds} تخت ---")
        rooms = self.data_structure.find_rooms_by_beds(beds)
        if not rooms:
            print("هیچ اتاقی با این تعداد تخت یافت نشد")
        else:
            for room in rooms:
                print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخته")
    except ValueError:
        print("لطفاً یک عدد صحیح وارد کنید")

def view_rooms_by_floor(self):
    try:
        floor = int(input("شماره طبقه: "))
        print(f"\n--- اتاق‌های طبقه {floor} ---")
        rooms = self.data_structure.find_rooms_by_floor(floor)
        if not rooms:
            print("هیچ اتاقی در این طبقه یافت نشد")
        else:
            for room in rooms:
                print(f"اتاق {room.room_id}: {room.beds} تخته")
    except ValueError:
        print("لطفاً یک عدد صحیح وارد کنید")

def display_inactive_rooms(self):
    inactive_rooms = self.data_structure.get_inactive_rooms()
    if not inactive_rooms:
        print("هیچ اتاق غیرفعالی وجود ندارد.")
        return

    print("\n--- اتاق‌های غیرفعال ---")
    for room in inactive_rooms:
        print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")

def change_room_status(self):
    try:
        room_id = int(input("شماره اتاق: "))
        room = self.data_structure.get_room(room_id)

        if not room:
            print("اتاق مورد نظر یافت نشد.")
            return

        current_status = "فعال" if room.active else "غیرفعال"
        print(f"وضعیت فعلی اتاق: {current_status}")

        if room.active:
            self.data_structure.deactivate_room(room_id)
            print("اتاق غیرفعال شد.")
        else:
            self.data_structure.activate_room(room_id)
            print("اتاق فعال شد.")
    except ValueError:
        print("شماره اتاق باید عدد باشد.")

def view_room_history(self):
    try:
        room_id = int(input("شماره اتاق: "))
        room = self.data_structure.get_room(room_id)
        if not room:
            print("اتاق مورد نظر یافت نشد.")
            return

        reservations = self.data_structure.get_room_reservations(room_id)
        if not reservations:
            print("هیچ رزروی برای این اتاق ثبت نشده است.")
            return

        print(f"\n--- تاریخچه رزرو اتاق {room_id} ---")
        for reservation in reservations:
            print(f"کد رزرو: {reservation.reservation_id}")
            print(f"کاربر: {reservation.user_id}")
            print(f"تاریخ ورود: {reservation.checkin}")
            print(f"تاریخ خروج: {reservation.checkout}")
            if reservation.is_cancelled:
                print(f"وضعیت: لغو شده - دلیل: {reservation.cancel_reason}")
            print("---")
    except ValueError:
        print("شماره اتاق باید عدد باشد.")

def view_reservations_by_date_range(self):
    start_date = input("تاریخ شروع (YYYY-MM-DD): ")
    end_date = input("تاریخ پایان (YYYY-MM-DD): ")

    if not self.policy_manager.validate_date_range(start_date, end_date):
        return

    reservations_queue = self.data_structure.get_reservations_in_range(start_date, end_date)
    if reservations_queue.is_empty():
        print("هیچ رزروی در این بازه زمانی یافت نشد.")
        return

    print("\n--- رزروهای بازه زمانی ---")
    while not reservations_queue.is_empty():
        _, reservation = reservations_queue.dequeue()
        print(f"کد رزرو: {reservation.reservation_id}")
        print(f"اتاق: {reservation.room_id}")
        print(f"کاربر: {reservation.user_id}")
        print(f"تاریخ ورود: {reservation.checkin}")
        print(f"تاریخ خروج: {reservation.checkout}")
        print("---")

def view_cancelled_reservations(self):
    cancelled_queue = self.data_structure.get_cancelled_reservations()

    if cancelled_queue.is_empty():
        print("هیچ رزرو لغو شده‌ای وجود ندارد.")
        return

    print("\n--- رزروهای لغو شده ---")
    while not cancelled_queue.is_empty():
        reservation_id, reason = cancelled_queue.dequeue()  # حالا reservation_id و reason را دریافت می‌کنیم
        print(f"کد رزرو: {reservation_id}")
        print(f"دلیل لغو: {reason}")
        print("---")

def view_blocked_users(self):
    blocked_users = self.data_structure.blocked_users.get_all_values()
    if blocked_users.is_empty():
        print("هیچ کاربر مسدودی وجود ندارد.")
        return

    print("\n--- کاربران مسدود شده ---")
    temp_queue = Queue()  # ایجاد یک صف موقت

    while not blocked_users.is_empty():
        user_id, block_date = blocked_users.dequeue()
        print(f"کد ملی: {user_id}")
        print(f"تاریخ مسدودیت: {block_date}")

        user = self.data_structure.get_user(user_id)
        if user:
            print("وضعیت: کاربر در سیستم ثبت شده است")
            print(f"تعداد رزروها: {user.reservations_count}")
            print(f"تعداد لغو رزروها: {user.cancelled_count}")
        else:
            print("وضعیت: کاربر در سیستم ثبت نشده است")
        print("---")

        # ذخیره در صف موقت
        temp_queue.enqueue(user_id, block_date)

    # بازگرداندن همه داده‌ها به هش تیبل
    while not temp_queue.is_empty():
        user_id, block_date = temp_queue.dequeue()
        self.data_structure.blocked_users.insert(user_id, block_date)

def room_service(self):
    cleaning_rooms = self.data_structure.get_cleaning_rooms()
    if not cleaning_rooms:
        print("هیچ اتاقی در لیست سرویس وجود ندارد.")
        return

    print("\n--- اتاق‌های نیازمند سرویس ---")
    for room in cleaning_rooms:
        status = "غیرفعال" if not room.active else "فعال"
        print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")
        print(f"وضعیت: {status}, روزهای رزرو شده: {room.service_counter}")

    try:
        room_id = int(input("\nشماره اتاق برای سرویس (0 برای انصراف): "))
        if room_id == 0:
            return

        if room_id not in [room.room_id for room in cleaning_rooms]:
            print("شماره اتاق نامعتبر است.")
            return

        success = self.data_structure.service_room(room_id)
        if success:
            print(f"اتاق {room_id} سرویس شد و از فردا فعال خواهد شد.")
            print("تعداد روزهای رزرو شده به صفر بازگشت.")
        else:
            print("خطا در انجام سرویس اتاق.")

    except ValueError:
        print("لطفاً یک شماره اتاق معتبر وارد کنید.")

def improve_room_service(self):
    result = self.data_structure.process_cleaning_queue()
    if result:
        room_id = result
        print(f"سرویس اتاق {room_id} انجام شد.")
    else:
        print("هیچ اتاقی در صف نظافت نیست.")

def go_to_next_day(self):
    current_date = input("تاریخ فعلی (YYYY-MM-DD): ")
    if self.policy_manager.validate_date(current_date):
        self.data_structure.process_date_change(current_date)
        print("تاریخ سیستم به‌روز شد و وضعیت‌های لازم بررسی شدند.")

# Adding methods to the class
AdminMenu.show_menu = show_menu
AdminMenu.create_room = create_room
AdminMenu.display_all_rooms = display_all_rooms
AdminMenu.display_all_customers = display_all_customers
AdminMenu.search_room = search_room
AdminMenu.view_rooms_by_beds = view_rooms_by_beds
AdminMenu.view_rooms_by_floor = view_rooms_by_floor
AdminMenu.display_inactive_rooms = display_inactive_rooms
AdminMenu.change_room_status = change_room_status
AdminMenu.view_room_history = view_room_history
AdminMenu.view_reservations_by_date_range = view_reservations_by_date_range
AdminMenu.view_cancelled_reservations = view_cancelled_reservations
AdminMenu.view_blocked_users = view_blocked_users
AdminMenu.room_service = room_service
AdminMenu.improve_room_service = improve_room_service
AdminMenu.go_to_next_day = go_to_next_day

In [ ]:
class CustomerMenu:
    def __init__(self, data_structure, file_manager, policy_manager):
        self.data_structure = data_structure
        self.file_manager = file_manager
        self.policy_manager = policy_manager

def show_menu(self, national_id):
    while True:
        print("\n--- پنل مشتری ---")
        print("1. جستجوی اتاق‌ها")
        print("2. نمایش مرتب اتاق‌ها")
        print("3. رزرو اتاق")
        print("4. مشاهده و لغو رزروها")
        print("5. نمایش تاریخچه رزرو")
        print("6. رزرو خودکار")
        print("7. رزرو سریع بهترین اتاق")
        print("8. تغییر اطلاعات حساب کاربری")
        print("9. خروج")

        choice = input("انتخاب کنید: ")

        try:
            if choice == "1":
                self.search_available_rooms()
            elif choice == "2":
                self.display_sorted_rooms()
            elif choice == "3":
                self.reserve_room(national_id)
            elif choice == "4":
                self.view_and_manage_reservations(national_id)
            elif choice == "5":
                self.view_reservation_history(national_id)
            elif choice == "6":
                self.auto_reserve_room(national_id)
            elif choice == "7":
                self.quick_reserve_best_room(national_id)
            elif choice == "8":
                self.update_customer_info(national_id)
            elif choice == "9":
                break
            else:
                print("انتخاب نامعتبر. لطفاً دوباره امتحان کنید.")
        except ValueError as e:
            print(f"خطا: {e}")
        except Exception as e:
            print(f"خطای غیرمنتظره: {e}")

def search_available_rooms(self):
    try:
        print("\n--- جستجوی اتاق‌ها ---")
        start_floor = int(input("طبقه شروع: "))
        end_floor = int(input("طبقه پایان: "))

        if start_floor > end_floor:
            start_floor, end_floor = end_floor, start_floor

        rooms = self.data_structure.find_rooms_in_floor_range(start_floor, end_floor)

        if rooms.is_empty():
            print("هیچ اتاقی در این محدوده یافت نشد.")
            return

        print("\n--- اتاق‌های یافت شده ---")
        while not rooms.is_empty():
            _, room = rooms.dequeue()
            print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")
            print(f"وضعیت: {'فعال' if room.active else 'غیرفعال'}")
            print("---")

    except ValueError:
        print("لطفاً عدد صحیح وارد کنید.")

def display_sorted_rooms(self):
    print("\n--- نمایش مرتب اتاق‌ها ---")
    print("1. نمایش براساس طبقات")
    print("2. نمایش براساس تعداد تخت")
    print("3. نمایش براساس طبقه و تعداد تخت")

    choice = input("انتخاب کنید: ")

    try:
        if choice == "1":
            floor = int(input("شماره طبقه: "))
            rooms = self.data_structure.get_sorted_rooms("floor", floor=floor)
        elif choice == "2":
            beds = int(input("تعداد تخت: "))
            rooms = self.data_structure.get_sorted_rooms("beds", beds=beds)
        elif choice == "3":
            floor = int(input("شماره طبقه: "))
            beds = int(input("تعداد تخت: "))
            rooms = self.data_structure.get_sorted_rooms("floor", floor=floor, beds=beds)
        else:
            print("انتخاب نامعتبر.")
            return

        if rooms.is_empty():
            print("هیچ اتاقی با این مشخصات یافت نشد.")
            return

        print("\n--- اتاق‌های یافت شده ---")
        while not rooms.is_empty():
            _, room = rooms.dequeue()
            print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")
            print("---")

    except ValueError:
        print("لطفاً مقادیر معتبر وارد کنید.")

def reserve_room(self, national_id):
    try:
        active_rooms = self.data_structure.get_active_rooms()
        if active_rooms.is_empty():
            print("هیچ اتاق فعالی برای رزرو وجود ندارد.")
            return

        print("\n--- اتاق‌های قابل رزرو ---")
        while not active_rooms.is_empty():
            _, room = active_rooms.dequeue()
            print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")
            print("---")

        room_id = int(input("\nشماره اتاق مورد نظر: "))
        checkin = input("تاریخ ورود (YYYY-MM-DD): ")
        checkout = input("تاریخ خروج (YYYY-MM-DD): ")

        checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
        checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
        days = (checkout_date - checkin_date).days

        if days < 1 or days > 7:
            print("مدت اقامت باید بین 1 تا 7 روز باشد")
            return

        if not self.policy_manager.validate_date_range(checkin, checkout):
            return

        if not self.data_structure.check_room_availability(room_id, checkin, checkout):
            print("این اتاق در تاریخ مورد نظر قابل رزرو نیست.")
            return

        is_valid, total_days = self.policy_manager.check_room_total_reserved_days(room_id, days)
        if not is_valid:
            print(f"خطا: با این رزرو، مجموع روزهای رزرو به {total_days} روز می‌رسد که بیشتر از حد مجاز است")
            return

        try:
            self.policy_manager.check_user_reservation_limit(national_id)
        except ValueError as e:
            print(f"خطا: {e}")
            return

        reservation_id = int(f"{national_id}{room_id}{datetime.now().strftime('%H%M%S')}")

        if self.data_structure.add_reservation(reservation_id, national_id, room_id, checkin, checkout):

            print(f"\nرزرو با موفقیت انجام شد. کد پیگیری: {reservation_id}")
        else:
            print("خطا در ثبت رزرو.")

    except ValueError as e:
        print(f"خطا: {e}")

def view_and_manage_reservations(self, national_id):
    try:
        print("\n--- مشاهده و لغو رزرو ---")
        print("1. مشاهده اطلاعات رزرو")
        print("2. لغو رزرو")

        choice = input("انتخاب کنید: ")

        if choice == "1":
            reservation_id = int(input("کد پیگیری را وارد کنید: "))

            # استفاده از queue برای دریافت تمام رزروها
            reservations_queue = self.data_structure.reservations.inorder_traversal()

            found = False
            while not reservations_queue.is_empty():
                key, reservation = reservations_queue.dequeue()

                if int(reservation.reservation_id) == reservation_id:
                    if str(reservation.user_id) == str(national_id):
                        print("\n--- اطلاعات رزرو ---")
                        print(f"کد پیگیری: {reservation.reservation_id}")
                        print(f"شماره اتاق: {reservation.room_id}")
                        print(f"تاریخ ورود: {reservation.checkin}")
                        print(f"تاریخ خروج: {reservation.checkout}")
                        if reservation.is_cancelled:
                            print("وضعیت: لغو شده")
                            print(f"دلیل لغو: {reservation.cancel_reason}")
                        else:
                            print("وضعیت: فعال")
                        found = True
                    else:
                        print("این رزرو متعلق به شما نیست.")
                    break

            if not found:
                print("رزروی با این کد پیگیری یافت نشد.")

        elif choice == "2":
            reservation_id = int(input("کد پیگیری رزرو مورد نظر برای لغو: "))
            reservations_queue = self.data_structure.reservations.inorder_traversal()

            found = False
            while not reservations_queue.is_empty():
                key, reservation = reservations_queue.dequeue()

                if int(reservation.reservation_id) == reservation_id:
                    if str(reservation.user_id) == str(national_id):
                        if reservation.is_cancelled:
                            print("این رزرو قبلاً لغو شده است.")
                        else:
                            reason = input("دلیل لغو رزرو: ")
                            try:
                                if self.data_structure.cancel_reservation(reservation_id, reason):

                                    print("رزرو با موفقیت لغو شد.")
                            except ValueError as e:
                                print(f"خطا: {e}")
                    else:
                        print("این رزرو متعلق به شما نیست.")
                    found = True
                    break

            if not found:
                print("رزروی با این کد پیگیری یافت نشد.")

    except ValueError as e:
        print(f"خطا: {e}")

def view_reservation_history(self, national_id):
    try:
        print("\n--- تاریخچه رزرو ---")
        reservation_code = input("کد پیگیری را وارد کنید: ")

        # بررسی رزروهای فعال
        all_reservations = self.data_structure.reservations.inorder_traversal()
        found = False

        while not all_reservations.is_empty():
            _, reservation = all_reservations.dequeue()
            if str(reservation.reservation_id) == str(reservation_code):
                print("\n=== اطلاعات رزرو ===")
                print(f"کد پیگیری: {reservation.reservation_id}")
                print(f"شماره اتاق: {reservation.room_id}")
                print(f"تاریخ ورود: {reservation.checkin}")
                print(f"تاریخ خروج: {reservation.checkout}")
                if not reservation.is_cancelled:
                    print("وضعیت: فعال")
                else:
                    print("وضعیت: لغو شده")
                    if hasattr(reservation, 'cancel_reason'):
                        print(f"دلیل لغو: {reservation.cancel_reason}")
                found = True
                break

        # اگر در رزروهای فعال پیدا نشد، در رزروهای لغو شده جستجو کن
        if not found:
            cancelled_reservations = self.data_structure.get_cancelled_reservations()
            while not cancelled_reservations.is_empty():
                _, cancelled = cancelled_reservations.dequeue()
                if str(cancelled.reservation_id) == str(reservation_code):
                    print("\n=== اطلاعات رزرو لغو شده ===")
                    print(f"کد پیگیری: {cancelled.reservation_id}")
                    print(f"شماره اتاق: {cancelled.room_id}")
                    print(f"تاریخ ورود: {cancelled.checkin}")
                    print(f"تاریخ خروج: {cancelled.checkout}")
                    print("وضعیت: لغو شده")
                    if hasattr(cancelled, 'cancel_reason'):
                        print(f"دلیل لغو: {cancelled.cancel_reason}")
                    found = True
                    break

        if not found:
            print("رزروی با این کد پیگیری یافت نشد.")

    except ValueError as e:
        print(f"خطا: {e}")
    except Exception as e:
        print(f"خطای غیرمنتظره: {e}")


def auto_reserve_room(self, national_id):
    try:
        beds = int(input("تعداد تخت مورد نظر: "))
        checkin = input("تاریخ ورود (YYYY-MM-DD): ")
        checkout = input("تاریخ خروج (YYYY-MM-DD): ")

        checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
        checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
        days = (checkout_date - checkin_date).days

        if days < 1 or days > 7:
            print("مدت اقامت باید بین 1 تا 7 روز باشد")
            return

        if not self.policy_manager.validate_date_range(checkin, checkout):
            return

        room_id, reservation_id = self.data_structure.auto_reserve_room(national_id, beds, checkin, checkout)

        if room_id and reservation_id:

            print(f"\nرزرو با موفقیت انجام شد.")
            print(f"اتاق: {room_id}")
            print(f"کد پیگیری: {reservation_id}")

            room = self.data_structure.get_room(room_id)
            room.active = False
            self.data_structure.rooms.update(room_id, room)
        else:
            print("هیچ اتاق مناسبی یافت نشد.")

    except ValueError as e:
        print(f"خطا: {e}")

def quick_reserve_best_room(self, national_id):
    try:
        beds = int(input("تعداد تخت مورد نظر: "))

        best_rooms = self.data_structure.get_best_rooms(beds)

        if best_rooms.is_empty():
            print("هیچ اتاقی با این مشخصات یافت نشد.")
            return

        print("\n--- بهترین اتاق‌های موجود ---")
        count = 0
        temp_rooms = Queue()
        while not best_rooms.is_empty() and count < 3:
            _, room = best_rooms.dequeue()
            print(f"اتاق {room.room_id}: طبقه {room.floor}, {room.beds} تخت")
            print(f"تعداد رزرو: {room.reservations_count}")
            print("---")
            temp_rooms.enqueue(room.room_id, room)
            count += 1

        room_id = int(input("\nشماره اتاق مورد نظر: "))
        checkin = input("تاریخ ورود (YYYY-MM-DD): ")
        checkout = input("تاریخ خروج (YYYY-MM-DD): ")

        checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
        checkout_date = datetime.strptime(checkout, "%Y-%m-%d")
        days = (checkout_date - checkin_date).days

        if days < 1 or days > 7:
            print("مدت اقامت باید بین 1 تا 7 روز باشد")
            return

        if not self.policy_manager.validate_date_range(checkin, checkout):
            return

        if not self.data_structure.check_room_availability(room_id, checkin, checkout):
            print("این اتاق در تاریخ مورد نظر قابل رزرو نیست.")
            return

        reservation_id = int(f"{national_id}{room_id}{datetime.now().strftime('%H%M%S')}")

        if self.data_structure.add_reservation(reservation_id, national_id, room_id, checkin, checkout):

            print(f"\nرزرو با موفقیت انجام شد. کد پیگیری: {reservation_id}")
        else:
            print("خطا در ثبت رزرو.")

    except ValueError as e:
        print(f"خطا: {e}")

def update_customer_info(self, national_id):
    try:
        print("\n--- تغییر رمز عبور ---")
        current_password = input("رمز عبور فعلی: ")
        user = self.data_structure.get_user(national_id)

        if not user or user.password != current_password:
            print("رمز عبور فعلی اشتباه است.")
            return

        new_password = input("رمز عبور جدید: ")
        if self.policy_manager.validate_password(new_password):
            user.password = new_password
            self.data_structure.users.update(national_id, user)

            print("رمز عبور با موفقیت تغییر کرد.")

    except ValueError as e:
        print(f"خطا: {e}")

# Adding methods to the class
CustomerMenu.show_menu = show_menu
CustomerMenu.search_available_rooms = search_available_rooms
CustomerMenu.display_sorted_rooms = display_sorted_rooms
CustomerMenu.reserve_room = reserve_room
CustomerMenu.view_and_manage_reservations = view_and_manage_reservations
CustomerMenu.view_reservation_history = view_reservation_history
CustomerMenu.auto_reserve_room = auto_reserve_room
CustomerMenu.quick_reserve_best_room = quick_reserve_best_room
CustomerMenu.update_customer_info = update_customer_info

In [ ]:
class Menu:
    def __init__(self, data_structure, file_manager, policy_manager):
        self.data_structure = data_structure
        self.file_manager = file_manager
        self.policy_manager = policy_manager

    def main_menu(self):
        while True:
            print("\n--- منوی اصلی ---")
            print("1. ورود به سیستم به‌عنوان مدیر")
            print("2. ورود به سیستم به‌عنوان مشتری")
            print("3. خروج")

            choice = input("انتخاب کنید: ")
            if choice == "1":
                self.admin_login_menu()
            elif choice == "2":
                self.customer_login_menu()
            elif choice == "3":
                print("خروج از سیستم...")
                break
            else:
                print("انتخاب نامعتبر. لطفاً دوباره امتحان کنید.")

    def admin_login_menu(self):
        while True:
            print("\n--- ورود به سیستم به‌عنوان مدیر ---")
            print("1. ورود")
            print("2. بازگشت به منوی اصلی")

            choice = input("انتخاب کنید: ")
            if choice == "1":
                username = input("نام کاربری: ")
                password = input("رمز عبور: ")
                if username == "admin" and password == "admin123":
                    self.admin_menu()
                else:
                    print("نام کاربری یا رمز عبور اشتباه است.")
            elif choice == "2":
                break
            else:
                print("انتخاب نامعتبر. لطفاً دوباره امتحان کنید.")

    def customer_login_menu(self):
        while True:
            print("\n--- ورود به سیستم به‌عنوان مشتری ---")
            print("1. ورود")
            print("2. ثبت‌نام")
            print("3. بازگشت به منوی اصلی")

            choice = input("انتخاب کنید: ")
            if choice == "1":
                national_id = input("کد ملی: ")
                password = input("رمز عبور: ")

                if self.data_structure.is_user_blocked(national_id):
                    print("حساب کاربری شما مسدود شده است.")
                    continue

                user = self.data_structure.get_user(national_id)
                if user and user.password == password:
                    self.customer_menu(national_id)
                else:
                    print("کد ملی یا رمز عبور اشتباه است.")
            elif choice == "2":
                self.register_customer()
            elif choice == "3":
                break
            else:
                print("انتخاب نامعتبر. لطفاً دوباره امتحان کنید.")

    def register_customer(self):
        national_id = input("کد ملی: ")
        if self.data_structure.get_user(national_id):
            print("این کد ملی قبلاً ثبت شده است.")
            return

        password = input("رمز عبور: ")
        if not self.policy_manager.validate_password(password):
            return

        try:
            self.data_structure.add_user(national_id, password)
            print("ثبت‌نام با موفقیت انجام شد.")
        except ValueError as e:
            print(f"خطا در ثبت‌نام: {e}")

    def admin_menu(self):
        admin = AdminMenu(self.data_structure, self.file_manager, self.policy_manager)
        admin.show_menu()

    def customer_menu(self, national_id):
        customer = CustomerMenu(self.data_structure, self.file_manager, self.policy_manager)
        customer.show_menu(national_id)

In [ ]:
from datetime import datetime, timedelta

class PolicyManager:
    def __init__(self, data_structure):
        self.data_structure = data_structure
        self.MIN_RESERVATION_DAYS = 1
        self.MAX_RESERVATION_DAYS = 7
        self.MAX_TOTAL_DAYS = 10
        self.MIN_CANCELLATION_DAYS = 1
        self.MAX_ACTIVE_RESERVATIONS = 3
        self.MAX_SERVICE_COUNT = 10
        self.MAX_CLEANING_DAYS = 3

    # متدهای مربوط به مشتری
    def validate_date(self, date_str):
        """اعتبارسنجی یک تاریخ"""
        try:
            date = datetime.strptime(date_str, "%Y-%m-%d")
            current_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            if date < current_date:
                raise ValueError("تاریخ نمی‌تواند در گذشته باشد")
            return True
        except ValueError as e:
            if "does not match format" in str(e):
                raise ValueError("فرمت تاریخ نامعتبر است (YYYY-MM-DD)")
            raise e

    def validate_date_range(self, checkin, checkout):
        """اعتبارسنجی بازه تاریخ"""
        try:
            checkin_date = datetime.strptime(checkin, "%Y-%m-%d")
            checkout_date = datetime.strptime(checkout, "%Y-%m-%d")

            if checkin_date >= checkout_date:
                raise ValueError("تاریخ ورود باید قبل از تاریخ خروج باشد")

            days = (checkout_date - checkin_date).days
            if days < self.MIN_RESERVATION_DAYS or days > self.MAX_RESERVATION_DAYS:
                raise ValueError(f"مدت اقامت باید بین {self.MIN_RESERVATION_DAYS} تا {self.MAX_RESERVATION_DAYS} روز باشد")

            self.validate_date(checkin)
            return True
        except ValueError as e:
            raise e

    def check_room_availability(self, room_id, checkin, checkout):
        """بررسی در دسترس بودن اتاق"""
        room = self.data_structure.get_room(room_id)
        if not room:
            raise ValueError("اتاق مورد نظر یافت نشد")

        if not room.active:
            raise ValueError("اتاق مورد نظر غیرفعال است")

        return self.data_structure.check_reservation_date_range(room_id, checkin, checkout)

    def check_user_reservation_limit(self, user_id):
        """بررسی محدودیت تعداد رزروهای کاربر"""
        user = self.data_structure.get_user(user_id)
        if not user:
            raise ValueError("کاربر یافت نشد")

        if self.data_structure.is_user_blocked(user_id):
            raise ValueError("کاربر مسدود شده است")

        active_reservations = self.data_structure.get_user_active_reservations(user_id)
        count = 0
        temp_queue = Queue()

        while not active_reservations.is_empty():
            priority, reservation = active_reservations.dequeue()
            temp_queue.enqueue(priority, reservation)
            if not reservation.is_cancelled:
                count += 1

        # بازگرداندن رزروها به صف اصلی
        while not temp_queue.is_empty():
            priority, reservation = temp_queue.dequeue()
            active_reservations.enqueue(priority, reservation)

        if count >= self.MAX_ACTIVE_RESERVATIONS:
            raise ValueError(f"شما نمی‌توانید بیش از {self.MAX_ACTIVE_RESERVATIONS} رزرو فعال همزمان داشته باشید")

        return True

    def can_cancel_reservation(self, reservation):
        """بررسی امکان لغو رزرو"""
        try:
            checkin_date = datetime.strptime(reservation.checkin, "%Y-%m-%d")
            current_date = datetime.now()

            days_until_checkin = (checkin_date - current_date).days
            if days_until_checkin < self.MIN_CANCELLATION_DAYS:
                raise ValueError("لغو رزرو باید حداقل یک روز قبل از تاریخ ورود انجام شود")

            return True
        except ValueError as e:
            raise e

    def validate_password(self, password):
        """اعتبارسنجی رمز عبور"""
        if len(password) < 8:
            raise ValueError("رمز عبور باید حداقل 8 کاراکتر باشد")

        has_digit = any(c.isdigit() for c in password)
        has_letter = any(c.isalpha() for c in password)

        if not (has_digit and has_letter):
            raise ValueError("رمز عبور باید شامل حداقل یک حرف و یک عدد باشد")

        return True

    def validate_national_id(self, national_id):
        """اعتبارسنجی کد ملی"""
        if not national_id.isdigit() or len(national_id) != 10:
            raise ValueError("کد ملی باید 10 رقم باشد")
        return True

    # متدهای مربوط به مدیر
    def check_room_total_reserved_days(self, room_id, new_days):
        """بررسی محدودیت مجموع روزهای رزرو"""
        total_days = new_days
        room_reservations = self.data_structure.get_room_reservations(room_id)
        temp_queue = Queue()

        while not room_reservations.is_empty():
            priority, reservation = room_reservations.dequeue()
            temp_queue.enqueue(priority, reservation)
            if not reservation.is_cancelled:
                checkin_date = datetime.strptime(reservation.checkin, "%Y-%m-%d")
                checkout_date = datetime.strptime(reservation.checkout, "%Y-%m-%d")
                total_days += (checkout_date - checkin_date).days

        # بازگرداندن رزروها به صف اصلی
        while not temp_queue.is_empty():
            priority, reservation = temp_queue.dequeue()
            room_reservations.enqueue(priority, reservation)

        if total_days > 30:  # محدودیت 30 روز در ماه
            return False, total_days
        return True, total_days

    def should_service_room(self, room_id):
        """بررسی نیاز اتاق به سرویس"""
        room = self.data_structure.get_room(room_id)
        if not room:
            raise ValueError("اتاق مورد نظر یافت نشد")

        return room.service_counter >= self.MAX_SERVICE_COUNT

    def process_service_status(self, room):
        """پردازش وضعیت سرویس اتاق"""
        if room.service_counter >= self.MAX_SERVICE_COUNT:
            room.active = False
            room.next_active_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
            return True
        return False

    def check_cleaning_status(self, room):
        """بررسی وضعیت نظافت اتاق"""
        if not room.last_cleaned:
            return True

        last_cleaned_date = datetime.strptime(room.last_cleaned, "%Y-%m-%d")
        days_since_cleaning = (datetime.now() - last_cleaned_date).days

        return days_since_cleaning <= self.MAX_CLEANING_DAYS

    def validate_room_id(self, room_id):
        """اعتبارسنجی شماره اتاق"""
        if not isinstance(room_id, int) or room_id < 1:
            raise ValueError("شماره اتاق نامعتبر است")
        return True

    def get_current_date(self):
        """دریافت تاریخ فعلی"""
        return datetime.now().strftime("%Y-%m-%d")

    def get_next_date(self):
        """دریافت تاریخ فردا"""
        return (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:
from datetime import datetime

def check_and_unblock_users(data_structure):
    current_date = datetime.now()
    blocked_queue = data_structure.blocked_users.get_all_values()
    temp_queue = Queue()  # صف موقت برای نگهداری داده‌ها

    while not blocked_queue.is_empty():
        user_id, blocked_until = blocked_queue.dequeue()
        if blocked_until:
            blocked_until_date = datetime.strptime(blocked_until, "%Y-%m-%d")
            if blocked_until_date > current_date:
                temp_queue.enqueue(user_id, blocked_until)

    # بازگرداندن داده‌های معتبر به هش تیبل
    while not temp_queue.is_empty():
        user_id, blocked_until = temp_queue.dequeue()
        data_structure.blocked_users.insert(user_id, blocked_until)

def main():
    try:
        data_structure = DataStructure()
        file_manager = FileManager("Test")
        policy_manager = PolicyManager(data_structure)

        print("\n--- بارگذاری داده‌ها از فایل‌ها ---")
        file_manager.load_all_data(data_structure)

        print("\n--- بررسی و رفع مسدودیت کاربران ---")
        try:
            check_and_unblock_users(data_structure)
            print("وضعیت مسدودیت کاربران بررسی و به‌روزرسانی شد.")
        except Exception as e:
            print(f"خطا در بررسی مسدودیت کاربران: {e}")

        menu = Menu(data_structure, file_manager, policy_manager)
        menu.main_menu()

        file_manager.save_all_data(data_structure)

    except Exception as e:
        print(f"خطای سیستمی: {e}")
        try:
            file_manager.save_all_data(data_structure)
        except:
            print("خطا در ذخیره‌سازی اضطراری داده‌ها")
    finally:
        print("\nپایان اجرای برنامه")

if __name__ == "__main__":
    main()

Checking directory: /content/Test


--- بارگذاری داده‌ها از فایل‌ها ---
--- بارگذاری داده‌ها از فایل‌ها ---
Loading users from: /content/Test/test_user.txt
Loading rooms from: /content/Test/test_room.txt
Loading room statistics from: /content/Test/test_room_stats.txt
Loading reservations from: /content/Test/test_reservation.txt
Loading blocked users from: /content/Test/test_blockuser.txt
Loading cancelled reservations from: /content/Test/test_cancelled.txt
همه اطلاعات با موفقیت بارگذاری شدند.

--- بررسی و رفع مسدودیت کاربران ---
وضعیت مسدودیت کاربران بررسی و به‌روزرسانی شد.

--- منوی اصلی ---
1. ورود به سیستم به‌عنوان مدیر
2. ورود به سیستم به‌عنوان مشتری
3. خروج
انتخاب کنید: 1

--- ورود به سیستم به‌عنوان مدیر ---
1. ورود
2. بازگشت به منوی اصلی
انتخاب کنید: 1
نام کاربری: admin
رمز عبور: admin123

--- پنل مدیر ---
1. ایجاد اتاق
2. نمایش تمام اتاق‌ها
3. نمایش مشتریان
4. جستجوی اتاق
5. مشاهده اتاق‌ها با تعداد تخت مشخص
6. مشاهده اتاق در یک طبقه مشخص
7. نمایش اتاق‌های غیر فعال
8. تغییر وضعیت